In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import json

# Your existing code
import speech_recognition as sr
import datetime

def load_translation_data():
    translation_data = {}
    with open("data6/english.txt", 'r', encoding='utf-8') as f_eng, open("data6/hindi.txt", encoding='utf-8') as f_hindi:
        for eng_line, hindi_line in zip(f_eng, f_hindi):
            eng_phrase = eng_line.strip().lower()
            hindi_translation = hindi_line.strip()
            translation_data[eng_phrase] = hindi_translation
    return translation_data

def translate_audio_to_hindi():
    translation_data = load_translation_data()
    recognizer = sr.Recognizer()
    now = datetime.datetime.now(datetime.timezone.utc).astimezone(datetime.timezone(datetime.timedelta(hours=5, minutes=30)))
    current_time = now.time()
    # Check if current time is after 6 PM IST
    if current_time < datetime.time(18, 0):
        print("Translation service is available only after 6 PM IST.")
        return
    with sr.Microphone() as source:
        print("Speak in English...")
        audio = recognizer.listen(source)
    try:
        spoken_text = recognizer.recognize_google(audio)
        print(f"Recognized: {spoken_text}")

        # Check if the recognized text is excluded (starts with 'M' or 'O')
        first_char = spoken_text.strip().split()[0][0].lower()
        if first_char in ['m', 'o']:
            print("Cannot translate words starting with 'M' or 'O'. Please try again.")
            return

        # Translate to Hindi using pre-loaded dataset
        spoken_text_lower = spoken_text.lower()
        if spoken_text_lower in translation_data:
            translated_text = translation_data[spoken_text_lower]
            print(f"Translated to Hindi: {translated_text}")
        else:
            print("Translation not found in the dataset.")
    except sr.UnknownValueError:
        print("Sorry, I couldn't understand the audio. Please repeat.")
        translate_audio_to_hindi()
    except sr.RequestError:
        print("Speech recognition service is unavailable.")
    except Exception as e:
        print(f"Error: {e}")

# TensorFlow model creation
def create_model(vocab_size, embedding_dim, max_length):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(vocab_size, activation='softmax')
    ])
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Prepare tokenizer and save it as JSON
def save_tokenizer(tokenizer, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(tokenizer.to_json())

# Prepare vocabulary JSON file
def save_vocab(vocab, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(vocab, f)

# Prepare vectorization JSON file
def save_vectorization(data, tokenizer, filename, max_length):
    sequences = tokenizer.texts_to_sequences(data)
    padded = pad_sequences(sequences, maxlen=max_length, padding='post')
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(padded.tolist(), f)

if __name__ == "__main__":
    translate_audio_to_hindi()
    translation_data = load_translation_data()
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(translation_data.keys())
    
    save_tokenizer(tokenizer, 'tokenizer.json')
    save_vocab(tokenizer.word_index, 'vocab.json')
    save_vectorization(list(translation_data.keys()), tokenizer, 'vectorization.json', max_length=50)
    
    vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
    embedding_dim = 64
    max_length = 50
    
    model = create_model(vocab_size, embedding_dim, max_length)
    model.summary()
    model.save('translation_model.h5')


Speak in English...
Recognized: good night
Translated to Hindi: शुभ रात्रि
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 50, 64)            1340096   
                                                                 
 bidirectional_5 (Bidirectio  (None, 128)              66048     
 nal)                                                            
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                                 
 dense_11 (Dense)            (None, 20939)             1361035   
                                                                 
Total params: 2,775,435
Trainable params: 2,775,435
Non-trainable params: 0
_________________________________________________________________
